In [34]:
import asyncio
import io
import glob
import os
import sys
import time
import uuid
import requests
from urllib.parse import urlparse
from io import BytesIO
from PIL import Image, ImageDraw
from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials
from azure.cognitiveservices.vision.face.models import TrainingStatusType, Person, SnapshotObjectType, OperationStatusType
import warnings
warnings.filterwarnings('ignore')

In [37]:
os.environ['FACE_SUBSCRIPTION_KEY'] = '5df507ddf5d84572b4c2c99d39879490'
os.environ['FACE_ENDPOINT'] = 'https://faceapiappzhaoqc.cognitiveservices.azure.com/'
remote_path = '/store/Data/AttendanceData/temp.png'
local_path = '/Users/zhaoqc/Program/Docker/age_gender_recognition/aragaki_yokogao.jpeg'

KEY = os.environ['FACE_SUBSCRIPTION_KEY']
ENDPOINT = os.environ['FACE_ENDPOINT']
print(KEY)
print(ENDPOINT)

5df507ddf5d84572b4c2c99d39879490
https://faceapiappzhaoqc.cognitiveservices.azure.com/


In [42]:
import paramiko

ssh = paramiko.SSHClient() 
ssh.load_host_keys(os.path.expanduser(os.path.join("~", ".ssh", "known_hosts")))
ssh.connect('10.1.103.21', username='zhaoqc', password='4425091zqc')
sftp = ssh.open_sftp()
sftp.chdir('/store/Data/AttendanceData/')
# (stdin, stdout, stderr) = ssh.exec_command('pwd')
# print(stdout.read().decode())
sftp.put(local_path,'./temp2.png')
sftp.close()
ssh.close()


FileNotFoundError: [Errno 2] No such file or directory: 'https://www.biography.com/.image/t_share/MTQ1MzAyNzYzOTgxNTE0NTEz/john-f-kennedy---mini-biography.jpg'

In [40]:
face_client = FaceClient(ENDPOINT, CognitiveServicesCredentials(KEY))

In [44]:
single_face_image_url = 'https://www.biography.com/.image/t_share/MTQ1MzAyNzYzOTgxNTE0NTEz/john-f-kennedy---mini-biography.jpg'
single_image_name = os.path.basename(single_face_image_url)
detected_faces = face_client.face.detect_with_url(url=single_face_image_url)
if not detected_faces:
    raise Exception('No face detected from image {}'.format(single_image_name))

# Display the detected face ID in the first single-face image.
# Face IDs are used for comparison to faces (their IDs) detected in other images.
print('Detected face ID from', single_image_name, ':')
for face in detected_faces: 
    print(face.face_id)
print()

# Save this ID for use in Find Similar
first_image_face_ID = detected_faces[0].face_id

def getRectangle(faceDictionary):
    rect = faceDictionary.face_rectangle
    left = rect.left
    top = rect.top
    bottom = left + rect.height
    right = top + rect.width
    return ((left, top), (bottom, right))



Detected face ID from john-f-kennedy---mini-biography.jpg :
4bc91d49-8dc3-4665-8885-64e7eb156cfa



In [51]:
print(dir(detected_faces[0]))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_attribute_map', '_classify', '_create_xml_node', '_flatten_subtype', '_get_rest_key_parts', '_infer_class_models', '_subtype_map', '_validation', 'additional_properties', 'as_dict', 'deserialize', 'enable_additional_properties_sending', 'face_attributes', 'face_id', 'face_landmarks', 'face_rectangle', 'from_dict', 'is_xml_model', 'recognition_model', 'serialize', 'validate']


In [50]:
# Detect the faces in an image that contains multiple faces
# Each detected face gets assigned a new ID
multi_face_image_url = "http://www.historyplace.com/kennedy/president-family-portrait-closeup.jpg"
multi_image_name = os.path.basename(multi_face_image_url)
detected_faces2 = face_client.face.detect_with_url(url=multi_face_image_url)
second_image_face_IDs = list(map(lambda x: x.face_id, detected_faces2))
similar_faces = face_client.face.find_similar(face_id=first_image_face_ID, face_ids=second_image_face_IDs)
print('Similar faces found in', multi_image_name + ':')
for face in similar_faces:
    first_image_face_ID = face.face_id
    # The similar face IDs of the single face image and the group image do not need to match, they are only used for identification purposes in each image.
    # The similar faces are matched using the Cognitive Services algorithm in find_similar().
    face_info = next(x for x in detected_faces2 if x.face_id == first_image_face_ID)
    if face_info:
        print('  Face ID: ', first_image_face_ID)
        print('  Face rectangle:')
        print('    Left: ', str(face_info.face_rectangle.left))
        print('    Top: ', str(face_info.face_rectangle.top))
        print('    Width: ', str(face_info.face_rectangle.width))
        print('    Height: ', str(face_info.face_rectangle.height))

Similar faces found in president-family-portrait-closeup.jpg:
  Face ID:  15019d2e-d7d0-4985-bd4b-845df39a9b73
  Face rectangle:
    Left:  413
    Top:  57
    Width:  55
    Height:  55


In [55]:
response = requests.get(multi_face_image_url)
img = Image.open(BytesIO(response.content))
draw = ImageDraw.Draw(img)

for face in similar_faces:
    face_info = next(x for x in detected_faces2 if x.face_id == first_image_face_ID)
    left_top = (face_info.face_rectangle.left,face_info.face_rectangle.top)
    bottom_right = (face_info.face_rectangle.left+face_info.face_rectangle.width,face_info.face_rectangle.top+face_info.face_rectangle.height)
    draw.rectangle((left_top,bottom_right), outline='red')
    
img.show()